In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q fasttext

     |████████████████████████████████| 68 kB 2.9 MB/s 


In [ ]:
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier, ExtraTreesClassifier
from sklearn import preprocessing

def get_model_num(model_name):
    """
    Return the model number based on the model name.
    Argument:
    model_name (str)
    """
    print(model_name)
    
    if model_name == 'DecisionTreeClassifier' or model_name.startswith("dt"):
        return 1
    elif model_name == 'NearestCentroid':
        return 2
    elif model_name == 'SVC':
        return 3
    elif model_name == 'RandomForestClassifier' or model_name.startswith('rf'):
        return 4
    elif model_name == 'XGBClassifier' or model_name.startswith('xgb'):
        return 8
    elif model_name.find('soft') != -1 or model_name.find('hard') != -1:
        return 9
    elif model_name == 'ExtraTreesClassifier' or model_name.startswith('et'):
        return 5
    elif model_name == 'AdaBoostClassifier' or model_name.startswith('ada'):
        return 6
    elif model_name == 'GradientBoostingClassifier' or model_name.startswith('gb'):
        return 7
    '''
    else:
        return model_name
    '''
    return 4

def get_model(model_num):
    """
    Return the model based on the model number.
    Argument:
    model_num (int)
    """
    if model_num == 1:
        return DecisionTreeClassifier()
    elif model_num == 2:
        return NearestCentroid()
    elif model_num==3:
        return svm.SVC(kernel = 'linear')
    elif model_num == 4:
        return RandomForestClassifier(n_estimators=100, max_features="auto",random_state=0)
    elif model_num == 8:
        return xgb.XGBClassifier(tree_method='gpu_hist', random_state=1,learning_rate=0.01)
    elif model_num == 5:
        return ExtraTreesClassifier(n_estimators=100, max_features="auto",random_state=0)
    elif model_num == 6:
        return AdaBoostClassifier(n_estimators=100)
    elif model_num == 7:
        return GradientBoostingClassifier(n_estimators=100)
    
   
def get_model_name(model_num):
    """
    Return the model based on the model number.
    Argument:
    model_num (int)
    """
    if model_num == 1:
        return 'DecisionTreeClassifier'
    elif model_num == 2:
        return 'NearestCentroid'
    elif model_num==3:
        return 'SVC'
    elif model_num == 4:
        return 'RandomForestClassifier'
    elif model_num == 5:
        return 'ExtraTreesClassifier'
    elif model_num == 6:
        return 'AdaBoostClassifier'
    elif model_num == 7:
        return 'GradientBoostingClassifier'
    elif model_num == 8:
        return 'XGBClassifier'
    else:
        return model_num
        
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Source: https://www.researchgate.net/post/Which_one_is_better_regarding_a_classification_problem_with_four_classes_Single_model_capable_of_nonlinear_classification_or_3_one-versus-rest_SVMs


def find_TP(y_true, y_pred, label):
    # counts the number of true positives (y_true = 1, y_pred = 1)
    sum_tp = 0
    for i in range(len(y_true)):
        if y_true[i] == label and y_pred[i] == label:
            sum_tp += 1
    return sum_tp

def find_FN(y_true, y_pred, label):
    # counts the number of false negatives (y_true = 1, y_pred = 0)
    sum_fn = 0
    for i in range(len(y_true)):
        if y_true[i] == label and y_pred[i] != label:
            sum_fn += 1
    return sum_fn

def find_FP(y_true, y_pred, label):
    # counts the number of false positives (y_true = 0, y_pred = 1)
    sum_fp = 0
    for i in range(len(y_true)):
        if y_true[i] != label and y_pred[i] == label:
            sum_fp += 1
    return sum_fp

def find_TN(y_true, y_pred, label):
    # counts the number of true negatives (y_true = 0, y_pred = 0)
    sum_tn = 0
    for i in range(len(y_true)):
        if y_true[i] != label and y_pred[i] != label:
            sum_tn += 1
    return sum_tn

def get_metrics(y_true, y_pred):
    """
    Return Accuracy, TP, TN, FP, FN, Sensitivity/TPR, Specificity/FPR, AUC, Precision, Recall, F1, Learning Curve - important.
    Arguments:
    y_test (dataframe): the target in the test dataset 
    y_pred (dataframe): the predicted labels
    """
    y_true = y_true.values
    print(y_true)
    print(y_pred)

    tn = []
    fp = []
    fn = []
    tp = []
    accuracy = []
    tpr = []
    tnr = []
    fpr = []
    fnr = []
    precision  = []
    recall = []
    f1 = []
    f1_macro = []
    i = 0
    while i <= 1:
        print(i)
        tn.append(find_TN(y_true, y_pred, i))
        print('tn: ', tn)
        fp.append(find_FP(y_true, y_pred, i))
        print('fp: ', fp)
        fn.append(find_FN(y_true, y_pred, i))
        print('fn: ', fn)
        tp.append(find_TP(y_true, y_pred, i))
        print('tp: ', tp)
        accuracy.append((tp[i] + tn[i])/(tp[i] + tn[i] + fp[i] + fn[i]))
        tpr.append(tp[i]/(fn[i] + tp[i]))
        tnr.append(tn[i]/(tn[i] + fp[i]))
        fpr.append(fp[i]/(fp[i] + tn[i]))
        fnr.append(fn[i]/(fn[i] + tp[i]))
        try:
            precision.append(tp[i]/(tp[i] + fp[i]))
        except ZeroDivisionError:
            precision.append(0)
        try:
            recall.append(tp[i]/(tp[i] + fn[i]))
        except ZeroDivisionError:
            recall.append(0)
        try:
            f1.append((2*precision[i]*recall[i])/(precision[i] + recall[i]))
        except ZeroDivisionError:
            f1.append(0)
        i += 1
    f1_macro = (2 * ((precision[0] + precision[1])/2) * ((recall[0] + recall[1])/2))/(precision[0] + precision[1] + recall[0] + recall[1])
    #TODO: fnr[1] = tpr[1]/recall[1] - tpr[1]
    return {'f1_macro': f1_macro, 'f1': f1[1], 'tp': tp[1], 'fp': fp[1], 'tn': tn[1],'fn': fn[1],'tpr': tpr[1], 'tnr': tnr[1], 'fpr': fpr[1], 'fnr': fnr[1], 'precision': precision[1], 'recall': recall[1], 'accuracy': accuracy[1]}

def get_perm_feature_metrics(y_true, y_pred):
    """
    Return Accuracy, TP, TN, FP, FN, Sensitivity/TPR, Specificity/FPR, AUC, Precision, Recall, F1, Learning Curve - important.
    Arguments:
    y_test (dataframe): the target in the test dataset 
    y_pred (dataframe): the predicted labels
    """
    y_true = y_true.values

    tn = []
    fp = []
    fn = []
    tp = []
    accuracy = []
    tpr = []
    tnr = []
    fpr = []
    precision  = []
    recall = []
    f1 = []
    f1_macro = []
    i = 0
    while i <= 1:
        tn.append(find_TN(y_true, y_pred, i))
        fp.append(find_FP(y_true, y_pred, i))
        fn.append(find_FN(y_true, y_pred, i))
        tp.append(find_TP(y_true, y_pred, i))
        accuracy.append((tp[i] + tn[i])/(tp[i] + tn[i] + fp[i] + fn[i]))
        tpr.append(tp[i]/(fn[i] + tp[i]))
        tnr.append(tn[i]/(tn[i] + fp[i]))
        fpr.append(fp[i]/(fp[i] + tn[i]))
        try:
            precision.append(tp[i]/(tp[i] + fp[i]))
        except ZeroDivisionError:
            precision.append(0)
        try:
            recall.append(tp[i]/(tp[i] + fn[i]))
        except ZeroDivisionError:
            recall.append(0)
        try:
            f1.append((2*precision[i]*recall[i])/(precision[i] + recall[i]))
        except ZeroDivisionError:
            f1.append(0)
        i += 1
    f1_macro = (2 * ((precision[0] + precision[1])/2) * ((recall[0] + recall[1])/2))/(precision[0] + precision[1] + recall[0] + recall[1])
    return f1_macro


def plot_lc(model, X, y, cv, resample = 0):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    train_sizes, train_scores, test_scores = learning_curve(estimator = model, X = X, y = y, train_sizes = np.linspace(0.01, 1.0, 50), cv = cv, scoring = 'f1_macro')

    # Create means and standard deviations of training set scores
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)

    # Create means and standard deviations of test set scores
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Draw lines
    plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
    plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

    # Draw bands
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

    # Create plot
    plt.title("Learning Curve")
    plt.xlabel("Training Set Size"), plt.ylabel("Macro-F1 Score"), plt.legend(loc="best")
    plt.tight_layout()
    plt.show()


def calc_auc(model, X_test, y_test, model_name, fig_path, resample = 0, balanced = 1):
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    #resample the training set (if applicable)
    #no skill prediction
    ns_probs_0 = [0 for _ in range(len(y_test))]
    ns_probs_1 = [1 for _ in range(len(y_test))]
    #model.fit(X_train, y_train)
    model_probs = model.predict_proba(X_test)
    model_probs = model_probs[:, 1]

    if balanced == 0:
        #TODO:
        # predict class values
        yhat = model.predict(X_test)
        # calculate precision and recall for each threshold
        precision, recall, _ = precision_recall_curve(y_test, model_probs)
        # calculate scores
        model_auc = auc(recall, precision)
        ns0 = len(y_test[y_test==0]) / len(y_test)
        ns1 = len(y_test[y_test==1]) / len(y_test)
        plt.plot(recall, precision, linestyle = '-', label=model_name)
        if model_name == "XGBClassifier":
            plt.plot([0, 1], [ns0, ns0], linestyle='--', label='No Skill 0')
            plt.plot([0, 1], [ns1, ns1], linestyle='-.', label='No Skill 1')
        # axis labels
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        # show the legend
        plt.legend()
        plt.savefig(fig_path + 'roc_curve.png')
        return {"ns0_auc": ns0, "ns1_auc": ns1, "model_auc": model_auc}
    else:
        #calculate roc curves
        ns0_fpr, ns0_tpr, _ = roc_curve(y_test, ns_probs_0)
        ns1_fpr, ns1_tpr, _ = roc_curve(y_test, ns_probs_1)
        model_fpr, model_tpr, _ = roc_curve(y_test, model_probs)
        
        if model_name == "XGBClassifier" or model_name.find("xgb") != -1:
            plt.plot(ns0_fpr, ns0_tpr, linestyle = '--', label = 'No Skill 0')
            plt.plot(ns1_fpr, ns1_tpr, linestyle = '-.', label = 'No Skill 1')
        
        plt.figure(get_model_num(model_name))
        plt.plot(model_fpr, model_tpr, linestyle = '-', label = model_name)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.savefig(fig_path + model_name + '_roc_curve.png')

        plt.figure(100)
        plt.plot(model_fpr, model_tpr, linestyle = '-', label = model_name)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.savefig(fig_path + 'roc_curve.png')


        ns0_auc = roc_auc_score(y_test, ns_probs_0)
        ns1_auc = roc_auc_score(y_test, ns_probs_1)
        model_auc = roc_auc_score(y_test, model_probs)
        #return {"ns0_auc": ns0_auc, "ns1_auc": ns1_auc, "model_auc": model_auc, "ns0_fpr": ns0_fpr, "ns0_tpr": ns0_tpr, "ns1_fpr": ns1_fpr, "ns1_tpr": ns1_tpr, "model_fpr": model_fpr, "model_tpr": model_tpr}
        return {"ns0_auc": ns0_auc, "ns1_auc": ns1_auc, "model_auc": model_auc}
        
        
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import json
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pickle
from numpy import arange

#from hummingbird.ml import convert

#from cuml import RandomForestClassifier as cuRF 
#from cuml.ensemble import RandomForestClassifier as cuRFC

def tune_clf(X, y, clf_type, path, type):
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
    max_depth = [int(x) for x in np.linspace(10, 500, num = 10)]
    #max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4]
    max_features = ['auto', 'sqrt', 'log2']
    bootstrap = [True, False]
    learning_rate = [x for x in arange(start = 0.05, stop = 1.0, step = 0.05)]
    min_child_weight = [1, 3, 5, 7, 9]
    colsample_bytree = [0.5, 0.75, 1.0]
    classifier = None
    filename = None
    donotwrite = 0
    # Create the grid
    if clf_type == 1: #RF
        #check if the model was already created
        if os.path.exists(path + 'rf_' + type + '_model'):
            classifier = pickle.load(open(path + 'rf_' + type + '_model', 'rb'))
            donotwrite = 1
        else:
            filename = "rf_" + type + "_model"
            grid = {'n_estimators' : n_estimators, 'max_depth' : max_depth, 'min_samples_split' : min_samples_split, 
                  'min_samples_leaf' : min_samples_leaf, 'max_features' : max_features, 'bootstrap': bootstrap}
            rf_clf = RandomForestClassifier()
            #rf_grid = GridSearchCV(rf_clf, grid, cv = 5)
            #classifier = rf_grid
            rf_random = RandomizedSearchCV(rf_clf, grid, cv = 5, n_iter = 100)
            classifier = rf_random
            classifier.fit(X, y)
            #TODO
    elif clf_type == 2: #Ada
        #check if the model was already created
        if os.path.exists(path + 'ada_' + type + '_model'):
            classifier = pickle.load(open(path + 'ada_' + type + '_model', 'rb'))
            donotwrite = 1
        else:
            filename = "ada_" + type + "_model"
            grid = {'n_estimators' : n_estimators, 'learning_rate' : learning_rate}
            ada_clf = AdaBoostClassifier()
            #ada_grid = GridSearchCV(ada_clf, grid, cv = 5)
            #classifier = ada_grid
            ada_random = RandomizedSearchCV(ada_clf, grid, cv = 5, n_iter = 100)
            classifier = ada_random
            classifier.fit(X, y)
    elif clf_type == 3: #XGB
        #check if the model was already created
        if os.path.exists(path + 'xgb_' + type + '_model'):
            classifier = pickle.load(open(path + 'xgb_' + type + '_model', 'rb'))
            donotwrite = 1
        else:
            filename = "xgb_" + type + "_model"
            grid = {'n_estimators' : n_estimators, 'learning_rate' : learning_rate, 'max_depth': max_depth, 'min_child_weight': min_child_weight, 'colsample_bytree' : colsample_bytree, 'eval_metric': ['auc']}
            xgb_clf = xgb.XGBClassifier()
            #xgb_grid = GridSearchCV(xgb_clf, grid, cv = 5)
            #classifier = xgb_grid
            xgb_random = RandomizedSearchCV(xgb_clf, grid, cv = 5, n_iter = 100)
            classifier = xgb_random
            classifier.fit(X, y)
    elif clf_type == 4: #ExtraTrees
        #check if the model was already created
        if os.path.exists(path + 'extratrees_' + type + '_model'):
            classifier = pickle.load(open(path + 'extratrees_' + type + '_model', 'rb'))
            donotwrite = 1
        else:
            filename = "extratrees_" + type + "_model"
            grid = {'n_estimators' : n_estimators, 'max_depth': max_depth}
            et_clf = ExtraTreesClassifier()
            #et_grid = GridSearchCV(et_clf, grid, cv = 5)
            #classifier = et_grid
            et_random = RandomizedSearchCV(et_clf, grid, cv = 5, n_iter = 100)
            classifier = et_random
            classifier.fit(X, y, eval_metric = 'auc')
    elif clf_type == 5: #GradientBoosting
        #check if the model was already created
        if os.path.exists(path + 'gb_' + type + '_model'):
            classifier = pickle.load(open(path + 'gb_' + type + '_model', 'rb'))
            donotwrite = 1
        else:
            filename = "gb_" + type + "_model"
            grid = {'n_estimators' : n_estimators, 'learning_rate' : learning_rate, 'max_depth': max_depth, 
                    'min_child_weight': min_child_weight,  'colsample_bytree' : colsample_bytree}
            gb_clf = GradientBoostingClassifier()
            #gb_grid = GridSearchCV(gb_clf, grid, cv = 5)
            #classifier = gb_grid
            gb_random = RandomizedSearchCV(gb_clf, grid, cv = 5, n_iter = 100)
            classifier = gb_random
            classifier.fit(X, y)
    if donotwrite == 0:
        open(path + 'best_params' + str(clf_type) + '.txt', 'a').write(str(classifier.best_params_))
        pickle.dump(classifier, open(path + filename, 'wb'))
    return classifier


def classify(path, X, y, extra = 1):
    #open the metrics file for writing
    metrics_file = open(path + 'ensemblemetrics.json', 'w')
    metrics_dict = dict()

    #print("one")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)
    #print("two")

    #X_train = df_to_tensor(X_train)
    #X_test = df_to_tensor(X_test)
    #y_train = df_to_tensor(y_train)
    #y_test = df_to_tensor(y_test)
    #X_train, X_test = X_train.to(device), X_test.to(device)
    #y_train, y_test = y_train.to(device), y_test.to(device)

    #Random forest, Ada Boost, XGB, Extra Trees and Gradient Boosting
    
    
    if extra == 0:
        clf = tree.DecisionTreeClassifier()
        #clf = convert(clf, 'pytorch')
        #clf.to(device)
        clf.fit(X_train, y_train)
        clf_dict={"dt":clf}
    else:   
    
        print("creating RF base classifier")
        #rf_clf = RandomForestClassifier()
        #rf_clf = convert(rf_clf, 'pytorch')
        #rf_clf.to(device)
        #print("training base classifier")
        #rf_clf.fit(X_train, y_train)
        tuned_rf = tune_clf(X_train, y_train, 1, path, type='grid')

        print("creating base XGB classifier")
        #xgb_clf = xgb.XGBClassifier()
        #xgb_clf = convert(xgb_clf, 'pytorch')
        #xgb_clf.to(device)
        #print("training base classifier")
        #xgb_clf.fit(X_train, y_train)
        tuned_xgb = tune_clf(X_train, y_train, 3, path, type = 'grid')
        
        print("creating AdaBoost classifier")
        ada_clf = AdaBoostClassifier()
        #ada_clf = convert(ada_clf, 'pytorch')
        #ada_clf.to(device)
        #ada_clf.fit(X_train, y_train)
        tuned_ada = tune_clf(X_train, y_train, 2, path, type = 'grid')

        print("creating ET classifier")
        #et_clf = ExtraTreesClassifier()
        #et_clf.to(device)
        #et_clf.fit(X_train, y_train)
        tuned_et = tune_clf(X_train, y_train, 4, path, type = 'grid')

        print("creating GB classifier")
        #gb_clf = GradientBoostingClassifier()
        #gb_clf = convert(gb_clf, 'pytorch')
        #gb_clf.to(device)
        #gb_clf.fit(X_train, y_train)
        tuned_gb = tune_clf(X_train, y_train, 5, path, type = 'grid')

        print("creating voting classifier")
        soft_voting_clf = VotingClassifier(estimators=[('tuned_rf', tuned_rf), ('tuned_xgb', tuned_xgb),
                                                       ('tuned_ada', tuned_ada), ('tuned_et', tuned_et), ('tuned_gb', tuned_gb)],
                                           voting='soft')
        #soft_voting_clf = VotingClassifier(estimators=[('rf', rf_clf), ('xgb', xgb_clf), ('ada', ada_clf), ('et', et_clf), ('gb', gb_clf)],
        #                                  voting='soft')
        #soft_voting_clf = convert(soft_voting_clf, 'pytorch')
        #soft_voting_clf.to(device)
        print("training voting classifier")
        soft_voting_clf.fit(X_train, y_train)
        pickle.dump(soft_voting_clf, open(path + '/' + 'tunedmodel', 'wb'))

        clf_dict = {'rf': tuned_rf, 'xgb': tuned_xgb, 'ada': tuned_ada, 
                    'et': tuned_et, 'gb': tuned_gb, 'soft': soft_voting_clf}
    
    for key, clf in clf_dict.items():
        #clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        metrics = get_metrics(y_test, y_pred)
        metrics_dict[key] = dict()
        for k, value in metrics.items():
            metrics_dict[key][k] = value
        print("Evaluating")
        metrics_auc_result = calc_auc(clf, X_test, y_test, key, path + 'Analysis/', resample = 0, balanced = 1)
        metrics_auc_path = path + 'Analysis/' + key + '_metrics_auc.json'
        with open(metrics_auc_path, 'w') as file:
            text = str(metrics_auc_result)
            file.write(text)
        #print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    #write the scores to the file
    json.dump(metrics_dict, metrics_file)
    metrics_file.close()
    
    
import pickle 

def test(X, y, path, analysis_path, folder_name):
    classifier = pickle.load(open(path + 'tunedmodel', 'rb'))
    y_pred = classifier.predict(X)
    
    metrics = get_metrics(y, y_pred)

    metrics_dict = dict()
    metrics_dict["model"] = dict()

    for k, value in metrics.items():
      metrics_dict["model"][k] = value

    auc_result = calc_auc(classifier, X, y,'soft', analysis_path + folder_name + '/', resample = 0, balanced = 1)

    metrics_auc_path = analysis_path + folder_name + '/' + 'test.json'

    metrics_file = open(analysis_path + folder_name + '/' + 'ensemblemetrics.json', 'w')


    with open(metrics_auc_path, 'w') as file:
        text = str(auc_result)
        file.write(text)
        json.dump(metrics_dict, metrics_file)
        metrics_file.close()


In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All+tuned/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All+tuned/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, 'test')


Index([                     0,                      1,                      2,
                            3,                      4,                      5,
                            6,                      7,                      8,
                            9,
       ...
                     'length',            'n_synsets',           'n_synonyms',
               'n_consonants',             'n_vowels',          'n_hypernyms',
                 'n_hyponyms', 'n_consonantconjuncts',          'n_syllables',
                  'frequency'],
      dtype='object', length=310) Index([                     0,                      1,                      2,
                            3,                      4,                      5,
                            6,                      7,                      8,
                            9,
       ...
                     'length',            'n_synsets',           'n_synonyms',
               'n_consonants',             'n_vowels',      

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

creating base XGB classifier
creating AdaBoost classifier


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

#All-len

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-len/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-len/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']


for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-synsets

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-synsets/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-synsets/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']


for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-synonyms

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-synonyms/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-synonyms/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']


for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-consonants

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-consonants/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-consonants/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']


for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-vowels

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-vowels/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-vowels/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']


for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-hypernyms

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-hypernyms/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-hypernyms/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-hyponyms

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-hyponyms/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-hyponyms/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-consonantconjuncts

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-cc/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-cc/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_syllables', 'frequency']

for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-syllables

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-syllables/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-syllables/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'frequency']

for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


#All-frequency

In [ ]:
#Source: https://www.kaggle.com/jungealexander/word2vec-and-random-forest-classification

import fasttext
import pandas as pd
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

file = '/content/drive/MyDrive/Colab/AI4Bharat/indicnlp.v1.hi.bin'
path = '/content/drive/MyDrive/Colab/Embeddings+All-freq/'
analysis_path = '/content/drive/MyDrive/Colab/Embeddings+All-freq/Analysis/'
#read the train and test files in majority dataset and insert the embeddings in the files
train_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/train_updated.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/CleanedData/RankedClassification/test_updated.csv')

model = fasttext.load_model(file)

def vectorise(key):
    try:
        return model.get_word_vector(key)
    except:
	      return [0]*300 #dimensions

X_train = train_df[['word']]
y_train = train_df['label']

X_test = test_df[['word']]
y_test = test_df['label']

#extract normalised embeddings
df = X_train.copy()
df['vector'] = X_train['word'].apply(vectorise)
X_train_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_train_vector)
X_train_vector =  pd.DataFrame(scaler.transform(X_train_vector))
train_df.index = X_train_vector.index

df = X_test.copy()
df['vector'] = X_test['word'].apply(vectorise)
X_test_vector = pd.DataFrame(df['vector'].values.tolist())
scaler = preprocessing.MinMaxScaler().fit(X_test_vector)
X_test_vector = pd.DataFrame(scaler.transform(X_test_vector))
test_df.index = X_test_vector.index

#columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables', 'frequency']

columns = ['length', 'n_synsets', 'n_synonyms', 'n_consonants', 'n_vowels', 'n_hypernyms', 'n_hyponyms', 'n_consonantconjuncts', 'n_syllables']

for column in columns:
    X_train_vector[column] = train_df[[column]]
    X_test_vector[column] = test_df[[column]]
print(X_train_vector.columns, X_test_vector.columns)

X_train_vector.to_csv(path + "trainembeddingsfreq.csv")
X_test_vector.to_csv(path + "testembeddingsfreq.csv")


classify(path, X_train_vector, y_train)
test(X_test_vector, y_test, path, analysis_path, folder_name = 'test')


In [ ]:
def test_bias(test_df, folder):
    X_test = test_df[['word']]
    y_test = test_df['label']
    df = X_test.copy()
    df['vector'] = X_test['word'].apply(vectorise)
    X_test_vector = pd.DataFrame(df['vector'].values.tolist())
    test(X_test_vector, y_test, analysis_path, folder)
'''
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_comfhi.csv'), 'test/bias/comfortablehi')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_comfnonhi.csv'), 'test/bias/comfortablenonhi')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_female.csv'), 'test/bias/female')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_male.csv'), 'test/bias/male')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_highfamiliarity.csv'), 'test/bias/hiacad')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_lowfamiliarity.csv'), 'test/bias/lowacad')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_native.csv'), 'test/bias/native')
test_bias(pd.read_csv('/content/drive/MyDrive/Colab/Bias/test_nonnative.csv'), 'test/bias/nonnative')
'''